In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
data= pd.read_csv('../input/data-train/train.tsv', sep="\t")

In [ ]:
data

In [ ]:
data.info()

In [ ]:
data.dtypes

In [ ]:
import seaborn as sns
sns.catplot(y="Sentiment", kind="count",
            palette="pastel", edgecolor=".6",
            data=data)

In [ ]:
data['Sentiment'].value_counts()

In [ ]:
y_train = data['Sentiment']

In [ ]:
first_class = y_train[y_train == 0]
second_class = y_train[y_train == 1]
third_class = y_train[y_train == 2]
forth_class = y_train[y_train == 3]
fifth_class = y_train[y_train == 4]

print('',len(first_class),'\n',len(second_class), '\n',len(third_class), '\n', len(forth_class), '\n', 
      len(fifth_class) )





***We can see clearly there is data-imbalance, between different classes we should solve this problem, in order to prevent the algorithm from being trained on (skewed dataset), we can make the data uniform, we reduce the number of samples to the length of first_class, which is minimum.***

In [ ]:
second_class = second_class[0:len(first_class)]
third_class  = third_class[0:len(first_class)]
forth_class  = forth_class[0:len(first_class)]
fifth_class  = fifth_class[0:len(first_class)]

***We filter split each class from the data-frame then we will reduce the size of each class to be the same after all***

In [ ]:
text_first_class  = data[['Phrase','Sentiment']]  [y_train==0]
text_second_class = data[['Phrase','Sentiment']]  [y_train==1]
text_third_class  = data[['Phrase','Sentiment']]  [y_train==2]
text_forth_class  = data[['Phrase','Sentiment']]  [y_train==3]
text_fifth_class  = data[['Phrase','Sentiment']]  [y_train==4]

print('',len(text_first_class),'\n',len(text_second_class), '\n',len(text_third_class), '\n', len(text_forth_class), '\n', 
      len(text_fifth_class) )

In [ ]:
text_second_class = text_second_class[0:len(text_first_class)]
text_third_class  = text_third_class [0:len(text_fifth_class)]
text_forth_class  = text_forth_class [0:len(text_first_class)]
text_fifth_class  = text_fifth_class [0:len(text_first_class)]

In [ ]:
frames = [text_first_class, text_second_class, text_third_class, text_forth_class, text_fifth_class]
new_train = pd.concat(frames)

In [ ]:
sns.catplot(y="Sentiment", kind="count",
            palette="pastel", edgecolor=".6",
            data=new_train)

In [ ]:
new_train

In [ ]:
X = new_train['Phrase'].values
type(X)


In [ ]:
y = new_train['Sentiment'].values

In [ ]:
y.shape,X.shape

In [ ]:
import tensorflow as tf
import keras

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
vocab_size = 10000
embedding_dim = 32
max_length = 150
trunc_type='post'
padding_type='post'
oov_tok = "<OOV>"

In [ ]:
tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(X)

word_index = tokenizer.word_index

training_sequences = tokenizer.texts_to_sequences(X)
training_padded = pad_sequences(training_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)



In [ ]:
import numpy as np
training_padded = np.array(training_padded)
training_padded.shape
y = y.reshape((37494,1))
y.shape
y

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(5, activation='softmax')
])
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

model.summary()

In [ ]:
num_epochs = 50

history = model.fit(training_padded, y, epochs=num_epochs)

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
acc = history.history['accuracy']
epochs = range(len(acc))

plt.plot(epochs, acc, 'r', label='Training accuracy')
plt.title('Training accuracy')
plt.legend()
plt.figure()

loss = history.history['loss']
plt.plot(epochs, loss, 'b', label='Training Loss')
plt.title('Training loss')
plt.legend()

plt.show()

In [ ]:
test_data = pd.read_csv('../input/datatest/test.tsv', sep="\t")

In [ ]:
test_data

In [ ]:
X_test = test_data['Phrase']
X_test.shape[0]

In [ ]:
X_test = X_test.reshape(X_test.shape[0],1)
X_test.shape
type(X_test)

In [ ]:
testing_sequences = tokenizer.texts_to_sequences(X_test)
testing_padded = pad_sequences(testing_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

In [ ]:
testing_padded.shape

In [ ]:
prediction = []

predictions = model.predict(testing_padded)

In [ ]:
predictions


In [ ]:
for i in predictions:
    prediction.append(np.argmax(i))

In [ ]:
submission =  pd.DataFrame({
        "PhraseId":test_data.PhraseId ,
        "Sentiment": prediction
    })

submission.to_csv('submission.csv', index=False)